# Understanding sales and the movement of products sold

## Introduction

A large part of a data scientists role consists of data engineering to implement analysis at scale. You should be prepared to analyze data in a variety of tools and languages, the larger the company - the more robust the tool or language. In a data science or analyst interview, you’ll be tested on your experience with the technical tools a company uses and your overall knowledge of programming theory.

**Business Context.** You work for AdventureWorks, a company that sells outdoor sporting equipment. The company has many different locations, products and customers and records all sales and customer focused data in their data lake. You have been given access to the relevant data files with documentation from the IT department. Your job is to extract meaningful insights from these data files to help increase sales and inventory turnover.

**Business Problem.** Leadership would like to to better inform our logistics team on total revenue and days of the week with the top revenues and products sold . Your task is to **write SQL queries that retrieve the answers to the above questions.**.

### Why SQL?:
SQL has remained a [top query language](https://www.indeed.com/career-advice/career-development/best-programming-languages-to-learn) and is heavily utilized in startups and established companies. As a data analyst or data scientist, you may encounter SQL, Python or R in a technical  interview. SQL is a primary language because in many roles, you may also be required to understand how to get data from source systems in the most efficient way. This interview could be a primary indicator of your on the job skills for many organizations.


### Frameworks for answering technical/whiteboard style questions:
During your interview you may be asked to answer questions conceptually (thinking out loud) or use a online coding environment such as [CoderPad](https://coderpad.io/), [HackerRank](https://www.hackerrank.com/), etc, to write and sometimes execute code. During this time your interviewer monitors and probes for deeper insight into why you chose certain functions. The interviewer is there to understand how efficient your solution is and in some cases, the number of errors or assistance you may require given a short time constraint. Use something like [SQLFiddle](http://sqlfiddle.com/) if you want to practice your SQL querying skills!

## Interview Questions

*SQL* - You are provided the following tables to answer the questions below: **SalesOrderDetail** (SalesOrderID, ProductID, OrderQty), **SalesOrderHeader** (SalesOrderID, OrderDate) and **Product** (ProductID, ListPrice)**:
1. Calculate total revenue
2. Calculate total/average/standard deviation of purchase quantities on a weekday (Monday - Friday), sorted by top number of purchases and display in table form.
3. Calculate total quantity of products that were not purchased on a weekday

**Questions 1 and 3 should display a single number, while question 2 should look similart to the table below:**

![Table sample of question 2 result](data/question2_answer.PNG)

## Overview of the data

The data for this case is contained in the [`AdventureWorks.db`](data/AdventureWorks.db) SQLite database. We will be focusing on the tables that belong to the Sales and Product categories. Complete documentation for the original data (of which you have only a subset) can be found [here](https://dataedo.com/download/AdventureWorks.pdf).

**Key Tables and Features:**
* **Product**: one row per product that the company sells
* **SalesOrderHeader**: one row per sale summarizing the sale

![Fields of the two tables](data/Tables.PNG)

The usual. Importing the libraries and the `sql` extension:

In [1]:
!pip install -r requirements.txt -q
import sqlalchemy
sqlite_engine = sqlalchemy.create_engine("sqlite:///AdventureWorks.db")

In [2]:
%load_ext sql
import pandas as pd

In [3]:
%sql sqlite:///AdventureWorks.db

Remember that in order to run SQL queries from within this notebook, you should write `%%sql` at the start of your code cell, like this:

In [4]:
%%sql
SELECT
    SalesOrderID
    , ProductID
    , OrderQty
FROM SalesOrderDetail
LIMIT 2;

 * sqlite:///AdventureWorks.db
Done.


salesorderid,productid,orderqty
43659,776,1
43659,777,3


In [5]:
%%sql
SELECT
    SalesOrderID
    , DATE(OrderDate)
FROM SalesOrderHeader
LIMIT 2;

 * sqlite:///AdventureWorks.db
Done.


salesorderid,DATE(OrderDate)
43659,2011-05-31
43660,2011-05-31


In [6]:
%%sql
SELECT 
    ProductID
    , ListPrice
FROM Product 
#ORDER BY 2 desc #ListPrice shows as 0 so we sort by descending to get an accurate representation of the data
LIMIT 10;

 * sqlite:///AdventureWorks.db
(sqlite3.OperationalError) near "#ORDER": syntax error
[SQL: SELECT ProductID , ListPrice
FROM Product 
#ORDER BY 2 desc #ListPrice shows as 0 so we sort by descending to get an accurate representation of the data
LIMIT 10;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [7]:
#Question 1: Total revenue
revenue = """
    SELECT sum(s.orderqty * p.ListPrice) as revenue
    FROM SalesOrderDetail s
    LEFT JOIN Product p ON s.ProductID = p.ProductID;
"""
revenue = pd.read_sql(revenue, con=sqlite_engine)
print('The total revenue is ${:,.2f}'.format(revenue.iloc[0][0]))

The total revenue is $170,964,699.59


Since we are using SQLite, we realized there is no Standard deviation command and we will need to get creative!! The formula for standard deviation: **Stdev = square root of variance**

We have square root as a SQLite command. So how do we calulate the variance?!!

\\(V(x) = E(x^2) - E(x)^2\\)

In [8]:
#Question 2: 
# Total number/average/standard deviation of purchase quantities for the set of weekdays (Monday - Friday) 
# ordered by descending number of purchases.

summary_stats = """
    SELECT strftime('%w', OrderDate) as weekday,
    sum (orderqty) as sum,
    avg (orderqty) as average,
    sqrt(avg(orderqty * orderqty) - avg(orderqty)*avg(orderqty)) as std_dev
    FROM SalesOrderDetail d
    LEFT JOIN SalesOrderHeader h ON h.SalesOrderID = d.SalesOrderID
    WHERE CAST(strftime('%w', OrderDate) AS INT) BETWEEN 1 AND 5
    GROUP BY 1
    ORDER BY 2 DESC;
"""
pd.read_sql(summary_stats, con=sqlite_engine)

,weekday,sum,average,std_dev
0,3,50546,2.496222,2.749487
1,1,44649,2.255114,2.310929
2,4,40248,2.377740,2.753052
3,2,37121,2.231097,2.375553
4,5,24263,1.840476,2.102788


In [9]:
# Question 3
# Number of item_ID’s that were NOT purchased in the weekdays.
not_weekday_sale = """
    SELECT COUNT(*) as non_weekday_sale_items
    FROM
    ( 
        SELECT ProductID
        FROM SalesOrderDetail d
        LEFT JOIN SalesOrderHeader h ON h.SalesOrderID = d.SalesOrderID
        WHERE CAST(strftime('%w', OrderDate) AS INT) NOT BETWEEN 1 AND 5
        GROUP BY 1
    );
"""
pd.read_sql(not_weekday_sale, con=sqlite_engine)

,non_weekday_sale_items
0,265


## Attribution

"AdventureWorks database", Nov 7, 2017, Microsoft Corporation, [MIT License](https://docs.microsoft.com/en-us/sql/samples/sql-samples-where-are?view=sql-server-ver15), https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/adventure-works